**Dcker test API**

In [3]:
import requests 
import json

In [4]:
url = "http://0.0.0.0:80/search"
query = "what is docker?"

In [5]:
params = {"query": query}
%time response = requests.get(url, params=params)

CPU times: user 2.41 ms, sys: 2.07 ms, total: 4.49 ms
Wall time: 37 ms


In [6]:
response.text

'{"title":["What is Docker? Docker container concept explained || Docker Tutorial 1","What is a Docker Container? Docker Demo || Docker Tutorial 2","Docker Tutorial for Beginners [FULL COURSE in 3 Hours]","Docker Crash Course for Absolute Beginners [NEW]","8 Basic Docker Commands || Docker Tutorial 4"],"video_id":["jPdIRX6q4jA","GeqaTjKMWeY","3c-iBn73dDE","pg19Z8LL06w","xGn7cFR3ARU"]}'

In [7]:
json.loads(response.text)['title']

['What is Docker? Docker container concept explained || Docker Tutorial 1',
 'What is a Docker Container? Docker Demo || Docker Tutorial 2',
 'Docker Tutorial for Beginners [FULL COURSE in 3 Hours]',
 'Docker Crash Course for Absolute Beginners [NEW]',
 '8 Basic Docker Commands || Docker Tutorial 4']

json.loads(response.text)['video_id']

**Testing With UI**

In [15]:
import gradio as gr
import requests

# Function to send request to the Docker API
def SearchResults(query):
    url = "http://0.0.0.0:80/search"  # Docker API endpoint
    params = {"query": query}

    try:
        response = requests.get(url, params=params)
        data = response.json()  # Parse the JSON response

        output_list = []
        num_empty_results = 5 - len(data['title'])

        # Display search results
        for i in range(len(data['title'])):
            video_id = data['video_id'][i]
            title = data['title'][i]

            embed = gr.HTML(value=f'<iframe width="576" height="324" src="https://www.youtube.com/embed/{video_id}"></iframe>', visible=True)
            text = gr.Markdown(value=f"""<br> <br>
            # {title}<br>
            🔗 [Video Link](https://youtu.be/{video_id})""", visible=True)

            output_list.append(embed)
            output_list.append(text)

        # If no results, display empty fields
        for i in range(num_empty_results):
            if num_empty_results == 5 and i == 0:
                embed = gr.HTML(visible=False)
                text = gr.Markdown(value="No results. Try rephrasing your query.", visible=True)

                output_list.append(embed)
                output_list.append(text)
                continue

            embed = gr.HTML(visible=False)
            text = gr.Markdown(visible=False)

            output_list.append(embed)
            output_list.append(text)

        return output_list

    except Exception as e:
        return [gr.Markdown(f"Error: {str(e)}")]


# Gradio UI
output_list = []

with gr.Blocks() as demo:
    gr.Markdown("# YouTube Search")

    with gr.Row():
        inputs = gr.Textbox(placeholder="What is docker?", label="Query", scale=3)
        btn = gr.Button("Search")
        btn.click(fn=SearchResults, inputs=inputs, outputs=output_list)

    for i in range(5):
        with gr.Row():
            output_list.append(gr.HTML())
            output_list.append(gr.Markdown())

    inputs.submit(fn=SearchResults, inputs=inputs, outputs=output_list)

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
